In [ ]:
import classes

# modified visualpriors library
from transforms import VisualPriorRepresentation, VisualPriorPredictedLabel
from taskonomy_network import TaskonomyEncoder, TaskonomyDecoder

import os
import collections
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import torchsummary

from PIL import Image
from scipy.stats import pearsonr
from scipy.io import savemat

import torch.utils.model_zoo # required to load nets
import torchvision.transforms.functional as TF
from torch import nn
import torchvision.models
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

#%load_ext autoreload
#%autoreload 2

# Analysis for all taskonomy models

In [ ]:
def run_taskonomy_encoder_integration_analysis(task, places_images):
    VisualPriorRepresentation._load_unloaded_nets([task])
    net = VisualPriorRepresentation.feature_task_to_net[task]

    _, eval_nodes = get_graph_node_names(net)
    return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
    net_tweaked = create_feature_extractor(net, return_nodes=return_nodes)

    integration_calc = classes.IntegrationCalculator(net_tweaked, return_nodes)
    return classes.calculate_dataset_integration(places_images, integration_calc)

#VisualPrior.viable_feature_tasks
tasks = ('autoencoding','depth_euclidean','jigsaw','reshading',
         'colorization', 'edge_occlusion','keypoints2d','room_layout',
         'curvature','edge_texture','keypoints3d','segment_unsup2d',
         'class_object','egomotion','nonfixated_pose','segment_unsup25d',
         'class_scene','fixated_pose','normal','segment_semantic',
         'denoising','inpainting','point_matching','vanishing_point')

places_images = classes.ImageDataset('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')

mdict = {}
for task in tasks:
    task_result = run_taskonomy_encoder_integration_analysis(task, places_images)
    task_result.to_csv(os.path.join('results', task, task + '.csv'))
    #mdict[task] = run_taskonomy_encoder_integration_analysis(task, places_images)



#savemat("taskonomy_integration")

# Linear modelling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut


X = StandardScaler().fit_transform(result)
y = StandardScaler().fit_transform(pd.DataFrame(places_images.beauty_ratings))
X.shape, y.shape
loo, predictions = LeaveOneOut(), []

for train_idx, predict_idx in loo.split(X):
    glm = LinearRegression().fit(X[train_idx], y[train_idx])
    predictions.append(glm.predict(X[predict_idx]).item())

predictions = pd.Series(predictions)

pearsonr(predictions, places_images.beauty_ratings)